In [9]:
from pprint import pprint
from rich import print

from CybORG import CybORG
from CybORG.Simulator.Scenarios import EnterpriseScenarioGenerator
from CybORG.Agents import SleepAgent
from CybORG.Simulator.Actions.AbstractActions import PrivilegeEscalate

steps = 5
sg = EnterpriseScenarioGenerator(blue_agent_class=SleepAgent, 
                                green_agent_class=SleepAgent, 
                                red_agent_class=SleepAgent,
                                steps=steps)
cyborg = CybORG(scenario_generator=sg, seed=1234)

cyborg.reset()

def agentDemographic()->dict:
  demographics = {}
  def countAgent(agents:list[str])->dict:
    numBlue = 0
    numRed = 0
    numGreen = 0
    for a in agents:
      if 'blue' in a:
        numBlue = numBlue+1
      if 'red' in a:
        numRed = numRed+1
      if 'green' in a:
        numGreen = numGreen+1
    return {'blue': numBlue, 'red': numRed, 'green':numGreen}

  demographics['total'] = countAgent(cyborg.agents)
  demographics['active'] = countAgent(cyborg.active_agents)
  return demographics
      
print(agentDemographic())

{'total': {'blue': 5, 'red': 6, 'green': 48}, 'active': {'blue': 5, 'red': 1, 'green': 48}}

In [15]:
reset = cyborg.reset(agent='red_agent_0')
first_session_host = list(reset.observation.keys())[1]
initial_obs = reset.observation

print("\nRed Agent 0: Initial Observation \n")
print(initial_obs)

Red Agent 0: Initial Observation

{
    'success': <TernaryEnum.UNKNOWN: 2>,
    'contractor_network_subnet_user_host_2': {
        'Interface': [
            {
                'interface_name': 'eth0',
                'ip_address': IPv4Address('10.0.179.198'),
                'Subnet': IPv4Network('10.0.179.0/24')
            }
        ],
        'Sessions': [
            {
                'session_id': 0,
                'username': 'ubuntu',
                'timeout': 0,
                'PID': 6873,
                'Type': <SessionType.RED_ABSTRACT_SESSION: 10>,
                'agent': 'red_agent_0'
            }
        ],
        'Processes': [{'PID': 6873, 'username': 'ubuntu'}],
        'User Info': [{'username': 'root', 'Groups': [{'GID': 0}]}, {'username': 'user', 'Groups': [{'GID': 1}]}],
        'System info': {
            'Hostname': 'contractor_network_subnet_user_host_2',
            'OSType': <OperatingSystemType.LINUX: 3>,
            'OSDistribution': <OperatingSystemDistribution.KALI: 9>,
            'OSVersion': <OperatingSystemVersion.K2019_4: 11>,
            'Architecture': <Architecture.x64: 2>,
            'position': array([0., 0.])
        }
    }
}

In [12]:
first_action = PrivilegeEscalate(hostname=first_session_host, session=0, agent='red_agent_0')
results = cyborg.step(agent='red_agent_0', action=first_action)
first_action_obs = results.observation

print("\nRed Agent 0: Observation #1 \n")
print(first_action_obs)

Red Agent 0: Observation #1

{
    'success': <TernaryEnum.TRUE: 1>,
    'contractor_network_subnet_user_host_0': {
        'Sessions': [
            {
                'session_id': 0,
                'username': 'root',
                'Type': <SessionType.RED_ABSTRACT_SESSION: 10>,
                'agent': 'red_agent_0'
            }
        ],
        'Interface': [{'ip_address': IPv4Address('10.0.22.237'), 'Subnet': IPv4Network('10.0.22.0/24')}],
        'System info': {'Hostname': 'contractor_network_subnet_user_host_0'}
    },
    'action': PrivilegeEscalate contractor_network_subnet_user_host_0
}